# SETUP

# TODO
- Aggiungere cella per diminuire calcoli (e dimensione dataset?)
- Creare un RDD i cui elementi siano le coppie righe/colonne e con un'operazione di map/reduce emettere coppie chiave-valore nelle quali la chiave indica la posizione (i, j) dell'elemento della matrice da aggiornare e il valore indica il nuovo elemento della matrice, utilizzando le formule del paragrafo 9.4.4. In questo modo "parallelizzate" gli aggiornamenti di tutti gli elementi della matrice. Eventualmente potete evitare di aggiornare d'un colpo tutta la matrice (potrebbe portare a difficoltà nella convergenza), decidendo in modo probabilistico ogni volta se aggiornare oppure no un elemento: basta che il job map/reduce generi un bit pseudocasuale che vale 1 con probabilità p, e se il suo valore è 1 allora utilizzate le formule di cui sopra, altrimenti restituite il valore attuale di U (fa parte della colonna che ottenete in input). In questo modo aggiornerete all'incirca una frazione pari a p dei valori della matrice. Lo stesso vale per V.

In [8]:
if "google.colab" in str(get_ipython()):
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [9]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


zsh:1: command not found: kaggle


unzip:  cannot find or open ./data/yelp-dataset.zip, ./data/yelp-dataset.zip.zip or ./data/yelp-dataset.zip.ZIP.


In [10]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

# Lavorare su un sample più piccolo (10%)
### conviente farlo solo sulle review per problemi di join

In [11]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


In [12]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    raw_df.show()
    #return raw_df.sample(frac=1/10, random_state = 0).dropna().rdd
    return raw_df.dropna().rdd

raw_review_RDD = from_json_to_RDD(review_filename).sample(True, 1/10, 0)  #699192 rows
raw_user_RDD = from_json_to_RDD(user_filename)
raw_business_RDD = from_json_to_RDD(business_filename)

raw_review_RDD.first()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

Row(business_id='04UD14gamNjLY0IDYVhHJg', cool=1, date='2015-09-23 23:10:31', funny=2, review_id='JrIxlS1TzJ-iCu79ul40cQ', stars=1.0, text="I am a long term frequent customer of this establishment. I just went in to order take out (3 apps) and was told they're too busy to do it. Really? The place is maybe half full at best. Does your dick reach your ass? Yes? Go fuck yourself! I'm a frequent customer AND great tipper. Glad that Kanella just opened. NEVER going back to dmitris!", useful=1, user_id='eUta8W_HdHMXPzLBBZhL1A')

# RETRIEVE RDDs

In [13]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),    # 0
            str(entry["user_id"]),      # 1
            str(entry["business_id"]),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')


There are 699192 reviews in the dataset
Reviews: ('JrIxlS1TzJ-iCu79ul40cQ', 'eUta8W_HdHMXPzLBBZhL1A', '04UD14gamNjLY0IDYVhHJg', 1, 1, 2, 1, "I am a long term frequent customer of this establishment. I just went in to order take out (3 apps) and was told they're too busy to do it. Really? The place is maybe half full at best. Does your dick reach your ass? Yes? Go fuck yourself! I'm a frequent customer AND great tipper. Glad that Kanella just opened. NEVER going back to dmitris!")


In [14]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (str(entry["user_id"]),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


There are 1987897 users in the dataset
Users: ('qVc8ODYU5SZjKXVBgXdI7w', 'Walker', 585, 3.91, 7217, 1259, 5994, 267)


In [15]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (str(entry["business_id"]),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD.map(get_business_tuple)

business_count = business_RDD.count()

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

There are 117618 business in the dataset
Business: ('mpf3x-BjTdTEA3yCZrAYPw', 'The UPS Store', 'Affton', 'MO', 3.0, 15, ['Shipping Centers', 'Local Services', 'Notaries', 'Mailbox Centers', 'Printing Services'])


# Calculate overall
### - for each business_id:

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{useful} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ useful} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [16]:
# TODO fix all comment

#review_RDD.filter(lambda x: x[3]==0).count()


#stars - useful - funny - cool - numero fan


#if stars <3
    #stars - {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2
#else
    #stars + {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2

    #dati mancanti: numero fan; best fan; best useful/funny/cool per ristorante
#rdd_test_ufc = (id ristorsante, best useful, best funny, best cool)
#best fan = query su dataset utenti --> user_RDD.max().first()

#review_id - user_id - id_rist - overall


In [ ]:
# get_overall_rating: function that generates the bias in order to incentivate or decrease the importance or overall rating of a review

# TODO fix function

#+ (fans/best_user_fans)) / 2

# def get_overall_rating(stars, best_useful, best_funny, best_cool, useful, funny, cool):
#     overall = (useful + (funny + cool)/2) / (best_useful + (best_funny + best_cool)/2) 
#     if stars < 3:
#         return stars - overall
#     else:
#         return stars + overall


In [19]:
def get_business_id_useful_funny_cool(row):
    """
        Writes a tuple with key business_id and attributes useful, funny, cool
    """
    return (row[2], (row[4], row[5], row[6]))

def get_max_useful_funny_cool(row1, row2):
    """
        Returns a new RDD with, for each business_id key, the maxima values of useful, funny, cool
    """
    return tuple(max(row1[i], row2[i]) for i in range(3))

review_tuple_RDD = review_RDD.map(get_business_id_useful_funny_cool)
review_best_ufc_RDD = review_tuple_RDD.reduceByKey(get_max_useful_funny_cool)

review_best_ufc_RDD.take(5)

# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 


('A2q7d-CBM2-81tVkmS4JMw', (12, 5, 8))

In [20]:
def rearrange_review_and_best_ufc_RDD(row):
    """
        Merge, for each business_id, the row values with the best useful, funny, cool ones
    """
    return (row[2], (row[0], row[1], row[3], row[4], row[5], row[6]))

review_ufc_RDD = review_RDD.map(rearrange_review_and_best_ufc_RDD).join(review_best_ufc_RDD)
review_ufc_RDD.first()

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))


('A2q7d-CBM2-81tVkmS4JMw',
 (('RB8UpF_kT2xoOC51OzXEeA', 'EZjT2qJN0mOXypMAqZdSrQ', 2, 1, 1, 0),
  (12, 5, 8)))

In [30]:
def partial_overall(x):
    """
        Rearrange review_ufc_RDD and calculates the partial overall of the delta above
        partial_overall ={[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2]
    """
    key = (x[1][0][0], x[1][0][1], x[0], x[1][0][2])
    num = (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2)
    den = (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2)
    return (*key,(num/den if sum(x[1][1]) != 0 else 0))


partial_review_overall_RDD = review_ufc_RDD.map(partial_overall)
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

('RB8UpF_kT2xoOC51OzXEeA',
 'EZjT2qJN0mOXypMAqZdSrQ',
 'A2q7d-CBM2-81tVkmS4JMw',
 2,
 0.08108108108108109)

In [31]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# TODO add map to turn the result in a simple tuple
# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


('NOq2SPYWZ35EvzxzwVt-kw',
 (('IpAAmIXb1YLnj-kjhMJZpg', 'A2q7d-CBM2-81tVkmS4JMw', 4, 0.05405405405405406),
  106))

In [33]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print("User with the highest number of fans: ",best_user_fans)

# TODO if best user fans = 0

def review_overall(x):
    user_id, review_id, business_id, stars= x[0], x[1][0][0], x[1][0][1], x[1][0][2]
    value = stars + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if stars >=3 else -2)
    return (review_id, user_id, business_id, value)

    #tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans)


review_overall_RDD = partial_overall_fans_RDD.map(review_overall)

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(5)


('37cpUoM8hlkSQfReIEBd-Q', 'Mike', 1882, 4.39, 22860, 10965, 15350, 12497)


[('wDbpA2oYJxpxqDHTJV09mQ',
  '37cpUoM8hlkSQfReIEBd-Q',
  'd4j4yUkJJojEVyqqOfGObw',
  6.0),
 ('C6E0t54Ct0xcxQgWZR3zuA',
  '37cpUoM8hlkSQfReIEBd-Q',
  'rgeuy1qbw6Z8B6CSVANHIA',
  5.8125),
 ('qqxEfslISv15fi0n9WOBqw',
  'hizGc5W1tBHPghM5YKCAtg',
  '7rU2gIQ1zyVUW9GNt2gVgQ',
  5.640813010808869),
 ('iNO80ME8rmNGvTvkeiHuFQ',
  'Hi10sGSZNxQH3NLyWSZ1oA',
  '9ycb4phY8qbaLcloDhmbNg',
  5.639753540849804),
 ('lkcVpI5Hbu7dzpEWJHJFUQ',
  'Hi10sGSZNxQH3NLyWSZ1oA',
  '7Cpp1cVc1mtC3PVicq9dLQ',
  5.639753540849804)]

# Basic Recommendation

In [34]:
#Keeping businesses that have at least 20 reviews

review_by_business_filtered_RDD = review_overall_RDD.map(lambda x: (x[2], (x[0], x[1], x[3]))).groupByKey()\
                                                    .mapValues(list).filter(lambda x: len(x[1])>=20)
# review_by_business_filtered_RDD.first()


In [35]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """
    #key = unicode('%.3f' % tuple[0])
    if (tuple[1][1]!= None):
        value = '{:.3f}'.format(tuple[1][1])
    else:
        value = ''
    key = tuple[0]
    return (value + ' ' + key)


In [37]:
#Make, for each business, the average of the stars for all the reviews

business_overall_RDD = review_by_business_filtered_RDD.map(lambda x: (x[0],  sum(i[2] for i in x[1])/len(x[1])))
business_overall_RDD.first()

('QGYzYUMsQe6k7__LD91E5w', 4.33020596914966)

In [36]:
# top recommended businesses
business_RDD.map(lambda x: (x[0], (x[1], x[6]))).leftOuterJoin(business_overall_RDD).sortBy(sortFunction, False).take(10)  #x[2], x[3],.filter(lambda x: 'Restaurants' in x[1][1])


[('jh8j-DWqgWkbRe_a2XtKFQ',
  (('Barrio Bread', ['Food', 'Specialty Food', 'Bakeries', 'Restaurants']),
   5.123154172385989)),
 ('elP0Xn9HyP6cRBuzbJdTpw',
  (('Urban Fresh',
    ['Vegan',
     'Live/Raw Food',
     'Food',
     'Juice Bars & Smoothies',
     'Restaurants',
     'Breakfast & Brunch']),
   5.109645984457759)),
 ('MnjCw3oaM0Hq39bjONDU0Q',
  (('Cheese Please',
    ['Beer',
     'Wine & Spirits',
     'Arts & Entertainment',
     'Wine Tasting Room',
     'Specialty Food',
     'Nightlife',
     'Cheese Shops',
     'Bars',
     'Food',
     'Wine Bars',
     'Wineries']),
   5.105063578895299)),
 ('2Sgt5ekJ7F785czScIZEYg',
  (('Piccola Gelateria',
    ['Restaurants',
     'Gelato',
     'Food',
     'Desserts',
     'Italian',
     'Creperies',
     'Internet Cafes',
     'Cafes',
     'Coffee & Tea']),
   5.099010688972281)),
 ('a3chKz0NwZcwCEhDqr9MpQ',
  (('The Studio Hair Artistry',
    ['Hair Extensions',
     'Waxing',
     'Makeup Artists',
     'Hair Stylists',
   

# Collaborative Filtering

In [ ]:
#TODO in the collaborative prt, take into consideration the categories and the State
#TODO based on the number of reviews, multiply by the percentage of the reviews made in a particular state (same for the categories)
# print(dict(sorted(business_RDD.flatMap(lambda x: tuple(x[6])).countByValue().items(), key=lambda x: x[1], reverse=True))) lista delle categorie più frequenti


## Preprocessing

In [ ]:
# Preprocessing
# Because the differences in the quality of items and the rating scales of users are
# such important factors in determining the missing elements of the matrix M , it
# is often useful to remove these influences before doing anything else. The idea
# was introduced in Section 9.3.1. We can subtract from each nonblank element
# mij the average rating of user i. Then, the resulting matrix can be modified
# by subtracting the average rating (in the modified matrix) of item j. It is also
# possible to first subtract the average rating of item j and then subtract the
# average rating of user i in the modified matrix. The results one obtains from
# doing things in these two different orders need not be the same, but will tend
# to be close. A third option is to normalize by subtracting from mij the average
# of the average rating of user i and item j, that is, subtracting one half the sum
# of the user average and the item average.
# If we choose to normalize M , then when we make predictions, we need to
# undo the normalization. That is, if whatever prediction method we use results
# in estimate e for an element mij of the normalized matrix, then the value
# we predict for mij in the true utility matrix is e plus whatever amount was
# subtracted from row i and from column j during the normalization process.



#We have chosen to implement the third option
#review_overall_RDD.flatMap(lambda x : ((x[1], x[2]), 1)).reduceByKey(lambda x, y : x + y).sortBy(lambda x: x[1], False)


In [ ]:
#media voti utente
average_user_RDD = review_overall_RDD.map(lambda x: (x[1], x[3])).reduceByKey(lambda x,y: (x+y)/2)
# average_user_RDD.take(10)


In [ ]:
#media voti business
average_business_RDD = review_overall_RDD.map(lambda x: (x[2], x[3])).reduceByKey(lambda x,y: (x+y)/2)
# average_business_RDD.take(10)


In [ ]:
#Merge review rates with average user
partial_normalized_RDD = review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3]))).join(average_user_RDD).map(lambda x: (x[1][0][0], x[0], x[1][0][1], x[1][0][2], x[1][1]))
# partial_normalized_RDD.take(10)

# tuple: (review_id, user_id, buisness_id, overall, average_user)


In [ ]:
normalized_RDD = partial_normalized_RDD.map(lambda x: (x[1], (x[0], x[2], x[3]))) \
                                       .join(average_business_RDD) \
                                       .map(lambda x: ((x[1][0][0], x[0]), (x[1][0][1] - (x[1][1]+x[1][0][2])/2))) \
                                       .reduceByKey(lambda x, y : (x+y)/2) \
                                       .map(lambda x: (*x[0], x[1]))

normalized_RDD.first()
# tuple: (user_id, buisness_id, overall - normalized_value)


## Initialization

In [ ]:
#We need to train and predict the vectors U and V
# train(trainingRDD, rank, iterations, regularization_parameter)  --> (U,V)
#We need to calculate RMSE between the utility and our matrix

rank = 2
# regularization_parameter = 0.1

U_RDD = user_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])

# U_RDD.take(5)
# V_RDD.take(5)


## Performing Optimization

In [ ]:
# (non-blank - (U x V))^2 + ...

# CV : 
# for (iter):
#     calc U
#     calc V
# RMSE 


In [ ]:
from functools import reduce
import operator

def update_U(entry, rank):
    id = entry[0]
    U_value = entry[1]
    res = 0

    all_review = normalized_RDD.filter(lambda x: x[1] == entry[0]).collect()
    if all_review != []:
        for review in all_review:
            V_value = V_RDD.lookup(review[2])
            res += review[3] - (reduce(operator.mul, U_value) + reduce(operator.mul, V_value))**2

    U_value[rank] = res
    return (id, U_value)

i = 0
U_RDD.map(lambda x :  update_U(x, i))


In [ ]:
#Update V